In [ ]:
import requests
import lxml.etree as ET
from teimporter.teimodule.tei import TeiPlaceList
from apis_core.helper_functions.RDFparsers import GenericRDFParser

In [ ]:
col, _ = Collection.objects.get_or_create(
    name='ASBW'
)
default_domain = "https://schnitzler-briefe.acdh.oeaw.ac.at"
pla_pla_rel, _ = PlacePlaceRelation.objects.get_or_create(name='hosts', name_reverse='located in')

In [ ]:
file = r"C:\Users\pandorfer\Documents\Redmine\konde\seh-data\hbas_places.xml"

In [ ]:
tei = TeiPlaceList(file)
listplace = tei.parse_placelist()
listplace['amount']

In [ ]:
errors = []
for x in listplace['places']:
    place_dict = tei.place2dict(x)
    legacy_id = "{}/{}".format(default_domain, place_dict['legacy_id'])
    pl = GenericRDFParser(legacy_id, 'Place').get_or_create()
    pl_type, _ = PlaceType.objects.get_or_create(name=place_dict['type'][0])
    pl.name = place_dict['placeNames'][0]['text']
    pl.kind = pl_type
    pl.collection.add(col)
    
    # create uris
    for uri in place_dict['idno']:
        try:
            domain = uri['subtype']
        except KeyError:
            domain = uri['type']
        if domain[0] == 'URL':
            url = uri['text']
        elif domain[0] == 'GEONAMES':
            url = "https://www.geonames.org/{}".format(uri['text'])
        elif domain[0] == 'WIENWIKI':
            url = uri['text'].strip()
        elif domain[0] == 'WIKIDATA':
            url = "https://www.wikidata.org/wiki/{}".format(uri['text'].strip())
        else:
            url = "{}/{}".format(default_domain, uri['text'])
        try:
            pl_uri, _ = Uri.objects.get_or_create(
                domain=domain,
                uri=url,
                entity=pl
            )
        except Exception as e:
            errors.append(e)
    
    # create labels
    for y in place_dict['placeNames'][1:]:
        try:
            label_str = y['type'][0]
        except IndexError:
            label_str = 'alt'
        label_type, _ = LabelType.objects.get_or_create(name=label_str)
        label, _ = Label.objects.get_or_create(
            label=y['text'], temp_entity=pl
        )
    
    # add coordinates
    
    coords = place_dict['geo']['coordinates']
    if coords:
        try:
            pl.lat = float(coords[0])
            pl.lng = float(coords[1])
        except ValueError:
            pass
    pl.save()

In [ ]:
place_dict

In [ ]:
for x in listplace['places']:
    place_dict = tei.place2dict(x)
    try:
        place_active = place_dict['belongs_to']['active']
    except KeyError:
        place_active = None
    try:
        place_passive = place_dict['belongs_to']['passive']
    except KeyError:
        place_passive = None
    if place_active and place_passive:
        try:
            pl_a = Uri.objects.get(uri="https://schnitzler-briefe.acdh.oeaw.ac.at/{}".format(place_active.strip())).entity.place
        except :
            print(place_active)
            pl_a = None
        try:
            pl_b = Uri.objects.get(uri="https://schnitzler-briefe.acdh.oeaw.ac.at/{}".format(place_passive.strip())).entity.place
        except:
            print(place_passive)
            pl_b = None
        if pl_a and pl_b:
            PlacePlace.objects.get_or_create(
                related_placeA=pl_a,
                related_placeB=pl_b,
                relation_type=pla_pla_rel
            )
        else:
            print('oje')
                

In [ ]:
Place.objects.all().delete()

In [ ]:
PlacePlace.objects.all().delete()

In [ ]:
relations = []
for x in ent_types:
    group = []
    for y in ent_dict['relations'][x]:
        rel = {}
        rel['rel_type'] = x
        rel['target'] = y[x[:-1]]['id']
        group.append(rel)
    if group:
        relations.append(group)


In [ ]:
for x in relations:
    print(x)

In [ ]:
file = "temp.xml"

In [ ]:
with open(file, 'wb') as f:
    f.write(ET.tostring(hansi.create_full_doc(), pretty_print=True, encoding='UTF-8'))

In [ ]:
hansi.export_full_doc()